# Predicting AI Acceptance in Mental Health Interventions through Self-Determination Theory

This notebook prepares **analysis-ready datasets** and performs **core EDA** for a cross-cultural study of AI acceptance in mental health interventions.

**Goals of this notebook:**
1. Load and harmonize **China** and **USA** survey datasets.
2. Compute all composite scales:
   - Self-Determination (TENS_Life_mean)
   - Epistemic Trust (ET_mean)
   - Stigma (SSRPH_mean)
   - Depression (PHQ5_mean)
   - General AI Attitudes (GAAIS_mean)
   - AI Acceptance (UTAUT_AI_mean)
3. Construct **intervention-specific acceptance scores**:
   - Accept_avatar (AI avatar / generic AI therapist)
   - Accept_chatbot (AI chatbot)
   - Accept_tele (teletherapy / human therapist)
4. Create merged datasets:
   - `data/merged/merged.csv`
   - `data/merged/intersection.csv`
   - `data/merged/union_with_suffix.csv`
5. Check data quality:
   - Descriptives by country
   - Reliability (Cronbach’s alpha)
   - Missingness patterns
   - Multiple imputation for key hypothesis variables

## 0.0. Setup & Paths
- Imports, plotting style, paths, KEY_COMPOSITES defined correctly.
- Key composites already includes the three intervention acceptances: Accept_avatar, Accept_chatbot, Accept_tele

In [ ]:
from __future__ import annotations

import warnings
from pathlib import Path
from typing import List, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# General plotting style
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

# Paths
DATA_DIR = Path("data")
MERGED_DIR = DATA_DIR / "merged"
CHINA_DIR = DATA_DIR / "china"
USA_DIR = DATA_DIR / "usa"

MERGED_DIR.mkdir(parents=True, exist_ok=True)

# Key composite variables used later for EDA and summaries
KEY_COMPOSITES = ["TENS_Life_mean", "ET_mean", 
"SSRPH_mean", "PHQ5_mean", 
"GAAIS_mean", "UTAUT_AI_mean", 
"Accept_avatar", "Accept_chatbot", "Accept_tele"]

# 1.0. China Data Processing

China has three separate files that need to be coalesced:
- CN_all.csv - Combined sample
- CN_client.csv - Client-specific items
- CN_therapist.csv - Therapist-specific items

## 1.1. Load Raw China Files

In [ ]:
cn_all = pd.read_csv(CHINA_DIR / "CN_all.csv")
cn_client = pd.read_csv(CHINA_DIR / "CN_client.csv")
cn_therapist = pd.read_csv(CHINA_DIR / "CN_therapist.csv")

print(f"CN_all: {cn_all.shape}")
print(f"CN_client: {cn_client.shape}")
print(f"CN_therapist: {cn_therapist.shape}")

In [ ]:
cn_all.head()

## 1.2. Merge Client/Therapist Extras into CN_all

### 1.2.1. Identify extra columns in client/therapist files

In [ ]:
# Find columns unique to client/therapist files
all_cols = set(cn_all.columns)
client_cols = set(cn_client.columns)
ther_cols = set(cn_therapist.columns)

# Columns that exist in client/therapist but not in all
missing_from_all = sorted(list((client_cols | ther_cols) - all_cols))

print(f"Found {len(missing_from_all)} columns to add from client/therapist files")
print(f"Examples: {missing_from_all[:5]}")

In [ ]:
# Get columns that overlap between client and therapist
overlap_cols = sorted(list(set(missing_from_all).intersection(client_cols).intersection(ther_cols)))
print(f"{len(overlap_cols)} columns exist in both client and therapist files")

### 1.2.2. Prepare extra columns from each files
We identify 53 columns present in client/therapist but missing from CN_all.
- client_extra (ID + client-only columns, renamed *_client)
- therapist_extra (ID + therapist-only columns, renamed *_therapist)
- We left-merge both onto cn_all by ID, ending with 620 columns.

In [ ]:
# Add suffixes to distinguish source
client_extra_cols = [c for c in missing_from_all if c in cn_client.columns]
therapist_extra_cols = [c for c in missing_from_all if c in cn_therapist.columns]

In [ ]:
# Create separate dataframes with just ID and the extra columns
client_extra = cn_client[["ID"] + client_extra_cols].copy()
therapist_extra = cn_therapist[["ID"] + therapist_extra_cols].copy()

In [ ]:
# Add suffixes to overlapping columns
client_extra = client_extra.rename(columns={c: f"{c}_client" for c in client_extra_cols})
therapist_extra = therapist_extra.rename(columns={c: f"{c}_therapist" for c in therapist_extra_cols})

In [ ]:
# Merge into cn_all
cn_augmented = cn_all.merge(client_extra, on="ID", how="left")
cn_augmented = cn_augmented.merge(therapist_extra, on="ID", how="left")

In [ ]:
print(f"Augmented CN_all from {cn_all.shape} to {cn_augmented.shape}")
cn_augmented.head()

## 1.3. Coalesce Client/Therapist Columns & Create role_label

### 1.3.1. Create unified variables and create role label

In [ ]:
# Create coalesced versions of overlapping columns
cn_coalesced = cn_augmented.copy()

# Get list of base names that have both _client and _therapist versions
client_suffix_cols = [c for c in cn_coalesced.columns if c.endswith("_client")]
therapist_suffix_cols = [c for c in cn_coalesced.columns if c.endswith("_therapist")]

# Find base names that exist in both
client_bases = {c.replace("_client", "") for c in client_suffix_cols}
therapist_bases = {c.replace("_therapist", "") for c in therapist_suffix_cols}
common_bases = sorted(list(client_bases & therapist_bases))

print(f"Found {len(common_bases)} variables with client/therapist versions")
print("Examples:", common_bases[:5])

### 1.3.3. Coalesce Data

In [ ]:
# For each paired column, create a unified version
for base in common_bases:
    client_col = f"{base}_client"
    therapist_col = f"{base}_therapist"
    
    # Coalesce: use client value if available, otherwise therapist value
    cn_coalesced[base] = cn_coalesced[client_col].fillna(cn_coalesced[therapist_col])

In [ ]:
# role_label based on the presence of client vs therapist data
sample_client_col = f"{common_bases[0]}_client"
sample_ther_col = f"{common_bases[0]}_therapist"

cn_coalesced["role_label"] = "unknown"
cn_coalesced.loc[cn_coalesced[sample_client_col].notna(), "role_label"] = "client"
cn_coalesced.loc[cn_coalesced[sample_ther_col].notna(), "role_label"] = "therapist"

print("Role distribution:")
print(cn_coalesced["role_label"].value_counts())

## 1.4. Compute Composite Scores for China

### 1.4.1. Self-Determination (TENS_Life_mean)

In [ ]:
# TENS_Life_mean (Self-Determination - 9 items, first 6 are reverse-coded)
tens_items = ['TENS_Life_1r', 'TENS_Life_2r', 'TENS_Life_3r', 'TENS_Life_4r', 'TENS_Life_5r', 
              'TENS_Life_6r', 'TENS_Life_7', 'TENS_Life_8', 'TENS_Life_9']

tens_available = [c for c in tens_items if c in cn_coalesced.columns]

cn_coalesced['TENS_Life_mean'] = cn_coalesced[tens_available].mean(axis=1)
print(f"TENS_Life_mean: {len(tens_available)}/9 items, Mean={cn_coalesced['TENS_Life_mean'].mean():.2f}")

### 1.4.2. Epistemic Trust (ET_mean)

In [ ]:
# ET_mean (Epistemic Trust - 15 items)
et_items = [f'ET_{i}' for i in range(1, 16)]

et_available = [c for c in et_items if c in cn_coalesced.columns]

cn_coalesced['ET_mean'] = cn_coalesced[et_available].mean(axis=1)

print(f"ET_mean: {len(et_available)}/15 items, Mean={cn_coalesced['ET_mean'].mean():.2f}")

### 1.4.3. Stigma (SSRPH_mean)

In [ ]:
# SSRPH_mean (Stigma - 5 items)
ssrph_items = [f'SSRPH_{i}' for i in range(1, 6)]

ssrph_available = [c for c in ssrph_items if c in cn_coalesced.columns]

cn_coalesced['SSRPH_mean'] = cn_coalesced[ssrph_available].mean(axis=1)

print(f"SSRPH_mean: {len(ssrph_available)}/5 items, Mean={cn_coalesced['SSRPH_mean'].mean():.2f}")

### 1.4.4. Depression (PHQ5_mean)

In [ ]:
# PHQ5_mean (Depression - 5 items)
phq_items = [f'PHQ5_{i}' for i in range(1, 6)]

phq_available = [c for c in phq_items if c in cn_coalesced.columns]

cn_coalesced['PHQ5_mean'] = cn_coalesced[phq_available].mean(axis=1)

print(f"PHQ5_mean: {len(phq_available)}/5 items, Mean={cn_coalesced['PHQ5_mean'].mean():.2f}")

### 1.4.5. General AI Attitude (GAAIS_mean)

In [ ]:
# GAAIS_mean (General AI Attitudes - using existing GAAIS_pos and GAAIS_neg if available)
cn_coalesced['GAAIS_mean'] = (cn_coalesced['GAAIS_pos'] + (8 - cn_coalesced['GAAIS_neg'])) / 2

# Overall is average of positive and (reversed) negative
print(f"GAAIS_mean: computed from pos/neg subscales, Mean={cn_coalesced['GAAIS_mean'].mean():.2f}")

### 1.4.6. AI Acceptance (UTAUT_AI_mean)

In [ ]:
# UTAUT_AI_mean (AI Acceptance - 26 items)
utaut_ai_items = [f'UTAUT_AI{i}' for i in range(1, 27)]
utaut_ai_items_with_r = []
for i in range(1, 27):
    if f'UTAUT_AI{i}r' in cn_coalesced.columns:
        utaut_ai_items_with_r.append(f'UTAUT_AI{i}r')
    elif f'UTAUT_AI{i}' in cn_coalesced.columns:
        utaut_ai_items_with_r.append(f'UTAUT_AI{i}')

if utaut_ai_items_with_r:
    cn_coalesced['UTAUT_AI_mean'] = cn_coalesced[utaut_ai_items_with_r].mean(axis=1)
    print(f"UTAUT_AI_mean: {len(utaut_ai_items_with_r)} items, Mean={cn_coalesced['UTAUT_AI_mean'].mean():.2f}")

## 1.5. UTAUT Subscales & Intervention-Specific Acceptance (China)

### 1.5.1. UTAUT subscales helper and intervention-specific acceptance

Helper function computes 7 UTAUT subscales for:
- UTAUT_AI (generic AI avatar / therapist),
- UTAUT_chatbot,
- UTAUT_human (human tele therapist).

Subscales are:
- EOU,
- SI,
- CONV,
- HC,
- PPR,
- HM,
- TQE

In [ ]:
def compute_utaut_subscales(df: pd.DataFrame, prefix: str = "UTAUT_AI") -> tuple[pd.DataFrame, Dict[str, int]]:
    """
    Compute UTAUT subscales for a given prefix (e.g., 'UTAUT_AI', 'UTAUT_chatbot', 'UTAUT_human').
    """
    subscale_mapping = {
        f"{prefix}_EOU_mean":  [1, 2, 3, 4],        # Ease of Use
        f"{prefix}_SI_mean":   [7, 8, 9],           # Social Influence
        f"{prefix}_CONV_mean": [10, 11, 12],        # Convenience
        f"{prefix}_HC_mean":   [13, 14, 22],        # Human Connection
        f"{prefix}_PPR_mean":  [16, 17, 18, 19, 20, 21],  # Privacy Risk (reversed)
        f"{prefix}_HM_mean":   [23, 24],            # Hedonic Motivation (reversed)
        f"{prefix}_TQE_mean":  [25, 26],            # Therapy Quality Expectancy
    }

    results: Dict[str, int] = {}

    for subscale_name, item_nums in subscale_mapping.items():
        cols = []
        for num in item_nums:
            if f"{prefix}{num}r" in df.columns:
                cols.append(f"{prefix}{num}r")
            elif f"{prefix}{num}" in df.columns:
                cols.append(f"{prefix}{num}")
        if cols:
            df[subscale_name] = df[cols].mean(axis=1)
            results[subscale_name] = len(cols)

    return df, results

In [ ]:
# Compute subscales for each UTAUT version in China data
utaut_versions = ["UTAUT_AI", "UTAUT_chatbot", "UTAUT_human"]
all_results = {}

for version in utaut_versions:
    version_cols = [c for c in cn_coalesced.columns if c.startswith(version)]
    if version_cols:
        cn_coalesced, results = compute_utaut_subscales(cn_coalesced, prefix=version)
        all_results[version] = results
        print(f"{version}: computed {len(results)} subscales")
        for sub, k in results.items():
            print(f"  {sub:28} k={k}, M={cn_coalesced[sub].mean():.2f}")
        print()

In [ ]:
# Recompute UTAUT_AI from subscales as a check
ai_subscales = [
    c for c in cn_coalesced.columns
    if c.startswith("UTAUT_AI_") and c.endswith("_mean")
    and any(f in c for f in ["EOU", "SI", "CONV", "HC", "PPR", "HM", "TQE"])
]
if ai_subscales:
    cn_coalesced["UTAUT_AI_mean_from_subscales"] = cn_coalesced[ai_subscales].mean(axis=1)
    corr = cn_coalesced[["UTAUT_AI_mean", "UTAUT_AI_mean_from_subscales"]].corr().iloc[0, 1]
    print(f"Recomputed UTAUT_AI_mean from {len(ai_subscales)} subscales.")
    print(f"Correlation with original: r = {corr:.3f}")

### 1.5.2. Intervention-Specific Acceptance (China)

In [ ]:
# Avatar / generic AI acceptance from UTAUT_AI subscales
if ai_subscales:
    cn_coalesced["Accept_avatar"] = cn_coalesced[ai_subscales].mean(axis=1)
    print(f"Accept_avatar (China): k={len(ai_subscales)}, "
          f"M={cn_coalesced['Accept_avatar'].mean():.2f}")

In [ ]:
# Chatbot acceptance from UTAUT_chatbot subscales
cb_subscales = [
    c for c in cn_coalesced.columns
    if c.startswith("UTAUT_chatbot_") and c.endswith("_mean")
    and any(k in c for k in ["EOU", "SI", "CONV", "HC", "PPR", "HM", "TQE"])
]
if cb_subscales:
    cn_coalesced["Accept_chatbot"] = cn_coalesced[cb_subscales].mean(axis=1)
    print(f"Accept_chatbot (China): k={len(cb_subscales)}, "
          f"M={cn_coalesced['Accept_chatbot'].mean():.2f}")

In [ ]:
# Teletherapy / human therapist acceptance from UTAUT_human subscales
human_subscales = [
    c for c in cn_coalesced.columns
    if c.startswith("UTAUT_human_") and c.endswith("_mean")
    and any(k in c for k in ["EOU", "SI", "CONV", "HC", "PPR", "HM", "TQE"])
]
if human_subscales:
    cn_coalesced["Accept_tele"] = cn_coalesced[human_subscales].mean(axis=1)
    print(f"Accept_tele (China): k={len(human_subscales)}, "
          f"M={cn_coalesced['Accept_tele'].mean():.2f}")

## 1.6.Add Country Label, Export China Data, China EDA


## 1.6.1. Add Country Label and Export China Data
- Adds Country = 'China'
- Saves CN_all_coalesced.csv (shape: 485 × 704)
- Shows all key composites have 485 valid scores (no missingness in China on these scales).

In [ ]:
# Add Country column
cn_coalesced['Country'] = 'China'

# Export to CSV
output_path = CHINA_DIR / "CN_all_coalesced.csv"
cn_coalesced.to_csv(output_path, index=False)

print("CHINA DATA COMPLETE")
print(f"Saved: {output_path}")
print(f"Shape: {cn_coalesced.shape}")
print(f"Participants: {len(cn_coalesced)}")
print(f"Variables: {len(cn_coalesced.columns)}")

### 1.6.2. Distribution Checks for Key Composite Variables

In [ ]:
available_composites_cn = [c for c in KEY_COMPOSITES if c in cn_coalesced.columns]

print("Key composite scores (China):")
for comp in available_composites_cn:
    n_valid = cn_coalesced[comp].notna().sum()
    mean = cn_coalesced[comp].mean()
    print(f"{comp:20} {n_valid:3} valid, M={mean:.2f}")

In [ ]:
# Distributions of main composites
fig, axes = plt.subplots(3, 3, figsize=(14, 10))
axes = axes.ravel()

for ax, comp in zip(axes, available_composites_cn):
    sns.histplot(cn_coalesced[comp], kde=True, ax=ax)
    ax.set_title(f"{comp} (China)")

for ax in axes[len(available_composites_cn):]:
    ax.axis("off")

plt.tight_layout()
plt.show()

# 2.0. USA Data Pipeline

## 2.1. Load USA Data & Standardize Names

Renaming:
- Age → age
- Gender → gender
- Edu → edu
- PHQ_5_i → PHQ5_i

In [ ]:
# Load USA data
usa_all = pd.read_csv(USA_DIR / "USA_all.csv", low_memory=False)
print(f"USA_all: {usa_all.shape}")

### 2.1.1 Standardize USA Column Names
- Aligns variable names China vs USA, crucial for later merging.
- Keeps this logic in one cell.

In [ ]:
# Create mapping dictionary for USA column renaming
usa_renamed = usa_all.copy()

# Rename demographics to match China
rename_map = {}
if 'Age' in usa_renamed.columns and 'age' not in usa_renamed.columns:
    rename_map['Age'] = 'age'
if 'Gender' in usa_renamed.columns and 'gender' not in usa_renamed.columns:
    rename_map['Gender'] = 'gender'
if 'Edu' in usa_renamed.columns and 'edu' not in usa_renamed.columns:
    rename_map['Edu'] = 'edu'

# Rename PHQ_5 to PHQ5
for i in range(1, 6):
    if f"PHQ_5_{i}" in usa_renamed.columns:
        rename_map[f"PHQ_5_{i}"] = f"PHQ5_{i}"
if "PHQ_5_mean" in usa_renamed.columns:
    rename_map["PHQ_5_mean"] = "PHQ5_mean"

# Apply renaming
usa_renamed.rename(columns=rename_map, inplace=True)
print(f"Renamed {len(rename_map)} columns (USA). Examples:", list(rename_map.items())[:5])

## 2.2 Compute Composite Scores for USA

### 2.2.1. Self-Determination (TENS_Life_mean)

In [ ]:
# TENS_Life_mean (needs reverse-coding 1–6; 7–9 as is)
tens_items_usa = [f"TENS_Life_{i}" for i in range(1, 10)]
tens_available_usa = [c for c in tens_items_usa if c in usa_renamed.columns]

usa_reversed = usa_renamed.copy()
for i in range(1, 7):
    col = f"TENS_Life_{i}"
    if col in usa_reversed.columns:
        usa_reversed[f"TENS_Life_{i}r"] = 8 - usa_reversed[col]

for i in range(1, 7):
    rcol = f"TENS_Life_{i}r"
    if rcol in usa_reversed.columns:
        usa_renamed[rcol] = usa_reversed[rcol]

tens_for_mean = [f"TENS_Life_{i}r" for i in range(1, 7)] + [f"TENS_Life_{i}" for i in range(7, 10)]
tens_for_mean = [c for c in tens_for_mean if c in usa_reversed.columns]
usa_renamed["TENS_Life_mean"] = usa_reversed[tens_for_mean].mean(axis=1)
print(f"TENS_Life_mean: {len(tens_available_usa)} items, "
      f"Mean={usa_renamed['TENS_Life_mean'].mean():.2f}")

### 2.2.2. Epistemic Trust (ET_mean)

In [ ]:
# ET_mean
et_items = [f'ET_{i}' for i in range(1, 16)]

et_available = [c for c in et_items if c in usa_renamed.columns]

usa_renamed['ET_mean'] = usa_renamed[et_available].mean(axis=1)

print(f"ET_mean: {len(et_available)}/15 items, Mean={usa_renamed['ET_mean'].mean():.2f}")

### 2.2.3. Stigma (SSRPH_mean)

In [ ]:
# SSRPH_mean
ssrph_items = [f'SSRPH_{i}' for i in range(1, 6)]

ssrph_available = [c for c in ssrph_items if c in usa_renamed.columns]

usa_renamed['SSRPH_mean'] = usa_renamed[ssrph_available].mean(axis=1)

print(f"SSRPH_mean: {len(ssrph_available)}/5 items, Mean={usa_renamed['SSRPH_mean'].mean():.2f}")

### 2.2.4. Depression (PHQ5_mean)

In [ ]:
# PHQ-5 mean
phq_items = [f'PHQ5_{i}' for i in range(1, 6)]

phq_available = [c for c in phq_items if c in usa_renamed.columns]

usa_renamed['PHQ5_mean'] = usa_renamed[phq_available].mean(axis=1)

print(f"PHQ5_mean: {len(phq_available)}/5 items, Mean={usa_renamed['PHQ5_mean'].mean():.2f}")

### 2.2.5. General AI Attitude

In [ ]:
# GAAIS_mean
usa_renamed['GAAIS_mean'] = (usa_renamed['GAAIS_pos'] + (8 - usa_renamed['GAAIS_neg'])) / 2
print(f"GAAIS_mean: computed from pos/neg subscales, Mean={usa_renamed['GAAIS_mean'].mean():.2f}")

### 2.2.6. AI Acceptance (UTAUT_AI_mean) from 86 items

In [ ]:
# UTAUT - USA has different structure (UTAUT_1_X, UTAUT_2_X, UTAUT_3_X)
# We'll average across all UTAUT items for simplicity
utaut_cols = [c for c in usa_renamed.columns if c.startswith('UTAUT_') and '_' in c[6:]]
utaut_cols = [c for c in utaut_cols if 'validation' not in c.lower()]

usa_renamed['UTAUT_AI_mean'] = usa_renamed[utaut_cols].mean(axis=1)
print(f"UTAUT_AI_mean: {len(utaut_cols)} items, Mean={usa_renamed['UTAUT_AI_mean'].mean():.2f}")

### Important Note on UTAUT Measurement

> [!WARNING]
> **Measurement Differences Across Countries:**
> - **China:** Uses a 26-item UTAUT_AI scale designed for AI acceptance in mental health
> - **USA:** Uses an 87-item composite UTAUT measure (includes multiple versions: AIavatar, etc.)
> - Both are treated as "AI Acceptance" (`UTAUT_AI_mean`) but represent **different instruments**
> - This measurement difference should be considered when interpreting cross-cultural comparisons
> - Subscale analyses may vary in validity across countries due to different item pools

## 2.3. Intervention-Specific Acceptance (USA)

### 2.3.1. Mapping
- UTAUT_1_* -> avatar AI
- UTAUT_2_* -> chatbot AI
- UTAUT_3_* -> teletherapy / human therapist

In [ ]:
avatar_cols_usa = [c for c in usa_renamed.columns if c.startswith("UTAUT_1_")]
chatbot_cols_usa = [c for c in usa_renamed.columns if c.startswith("UTAUT_2_")]
human_cols_usa = [c for c in usa_renamed.columns if c.startswith("UTAUT_3_")]

if avatar_cols_usa:
    usa_renamed["Accept_avatar"] = usa_renamed[avatar_cols_usa].mean(axis=1)
    print(f"Accept_avatar (USA): k={len(avatar_cols_usa)}, M={usa_renamed['Accept_avatar'].mean():.2f}")

if chatbot_cols_usa:
    usa_renamed["Accept_chatbot"] = usa_renamed[chatbot_cols_usa].mean(axis=1)
    print(f"Accept_chatbot (USA): k={len(chatbot_cols_usa)}, M={usa_renamed['Accept_chatbot'].mean():.2f}")

if human_cols_usa:
    usa_renamed["Accept_tele"] = usa_renamed[human_cols_usa].mean(axis=1)
    print(f"Accept_tele (USA): k={len(human_cols_usa)}, M={usa_renamed['Accept_tele'].mean():.2f}")

- USA is roughly similar to China for avatar/chatbot acceptance (≈3.7)
- USA is much lower on teletherapy (≈2.4 vs China ≈3.7).

## 2.4. Country Label, role_label Placeholder, USA EDA
- Country = 'USA' added.
- role_label is temporarily set to "unknown" for everyone because you don’t yet have a coded role variable in the US survey.

### 2.4.1. Add Country Label for USA

In [ ]:
# Add Country column
usa_renamed['Country'] = 'USA'

print("USA DATA COMPLETE")
print(f"Shape: {usa_renamed.shape}")
print(f"Participants: {len(usa_renamed)}")
print(f"Variables: {len(usa_renamed.columns)}")

In [ ]:
# Start from existing usa_renamed DataFrame
usa_renamed["role_label_usa3"] = "community"  # default category

### 2.4.2. Define clinician vs patient vs community roles

In [ ]:
# 1) Identify therapist / clinician flag column
therapist_flag_candidates = [
    "therapistfilterNEWUSETHIS",
    "Therapistfilter_new2USETHISforUTAUTvalidation",
]

therapist_flag_col = None
for col in therapist_flag_candidates:
    if col in usa_renamed.columns:
        therapist_flag_col = col
        break

In [ ]:
# 2) Identify patient flag column
patient_flag_candidates = ["Patientfilter_new2USETHISforallstudies"]

patient_flag_col = None
for col in patient_flag_candidates:
    if col in usa_renamed.columns:
        patient_flag_col = col
        break

In [ ]:
print("Therapist flag column used:", therapist_flag_col)
print("Patient flag column used:", patient_flag_col)

In [ ]:
# 3) Assign clinicians (therapists) – clinicians override patient status if both
if therapist_flag_col is not None:
    usa_renamed.loc[usa_renamed[therapist_flag_col] == 1, "role_label_usa3"] = "clinician"

In [ ]:
# 4) Assign patients (only if not already classified as clinician)
if patient_flag_col is not None:
    usa_renamed.loc[
        (usa_renamed[patient_flag_col] == 1)
        & (usa_renamed["role_label_usa3"] != "clinician"),
        "role_label_usa3"
    ] = "patient"

In [ ]:
# 5) Inspect final 3-level role distribution
print("USA 3-level role distribution (clinician / patient / community):")
print(usa_renamed["role_label_usa3"].value_counts(dropna=False))

### 2.4.3. USA Key Composites Summaries

In [ ]:
available_composites_usa = [c for c in KEY_COMPOSITES if c in usa_renamed.columns]
print("Key composite scores (USA):")
for comp in available_composites_usa:
    n_valid = usa_renamed[comp].notna().sum()
    mean = usa_renamed[comp].mean()
    print(f"{comp:20} {n_valid:4} valid, M={mean:.2f}")

In [ ]:
# Visual comparison
tmp = pd.concat([
    cn_coalesced[["Country"] + KEY_COMPOSITES],
    usa_renamed[["Country"] + KEY_COMPOSITES]
], ignore_index=True)

fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.ravel()

for ax, comp in zip(axes, KEY_COMPOSITES):
    sns.boxplot(data=tmp, x="Country", y=comp, ax=ax)
    ax.set_title(comp)

plt.tight_layout()
plt.show()

# 3.0. Cross-Country Merge & Dataset Variants

Create a single merged dataset with columns that exist in both countries.

## 3.1. Identify Common & Country-Specific Columns, Build Merged/Intersection/Union
- common_cols = 273 variables shared by China & USA.
- key_vars (including all composites + Accept_*) are confirmed present in common columns (15/15).

We build:

- intersection (strict common columns)

- merged (same as intersection; you reindex and concat, but shapes match: 2342×273)

- union_with_suffix (all variables with _CN/_USA suffixes)

In [ ]:
# Find common columns
cn_cols = set(cn_coalesced.columns)
usa_cols = set(usa_renamed.columns)

common_cols = sorted(list(cn_cols & usa_cols))
cn_only = sorted(list(cn_cols - usa_cols))
usa_only = sorted(list(usa_cols - cn_cols))

print(f"Common columns: {len(common_cols)}")
print(f"China-only columns: {len(cn_only)}")
print(f"USA-only columns: {len(usa_only)}")

In [ ]:
# Ensure key variables are in common columns
key_vars = [
    "Country", "age", "gender", "edu",
    "TENS_Life_mean", "ET_mean", "SSRPH_mean",
    "PHQ5_mean", "GAAIS_mean", "UTAUT_AI_mean",
    "GAAIS_pos", "GAAIS_neg",
    "Accept_avatar", "Accept_chatbot", "Accept_tele",
]

key_vars_available = [v for v in key_vars if v in common_cols]

print(f"Key variables in common: {len(key_vars_available)}/{len(key_vars)}")
print(f"Available: {key_vars_available}")

### 3.1.1. Intersection dataset: strict common columns

In [ ]:
intersection_cols = sorted(common_cols)

cn_intersection = cn_coalesced[intersection_cols].copy()
usa_intersection = usa_renamed[intersection_cols].copy()

intersection = pd.concat([cn_intersection, usa_intersection], ignore_index=True)
print(f"Intersection dataset: {intersection.shape}")

### 3.1.2. Merged dataset

It contains
- Three intervention-specific outcomes
- Key confounders
- Main predictor (TENS_Life_mean)
- Moderator (Country)
- China: role_label (client / therapist)
- USA: role_label_usa3 (clinician / patient / community)

In [ ]:
merge_cols = intersection_cols + ["role_label", "role_label_usa3"]

cn_subset = cn_coalesced.reindex(columns=merge_cols)
usa_subset = usa_renamed.reindex(columns=merge_cols)

merged = pd.concat([cn_subset, usa_subset], ignore_index=True)
print(f"Merged dataset: {merged.shape}")
print("Country distribution:")
print(merged["Country"].value_counts().to_string())

In [ ]:
print("Role columns in merged:")
print([c for c in merged.columns if "role" in c.lower()])

### 3.1.3. Union with suffixes: preserves all columns

In [ ]:
# Add country-specific suffixes only to the non-shared columns
cn_union = cn_coalesced.rename(columns={c: f"{c}_CN" for c in cn_only})
usa_union = usa_renamed.rename(columns={c: f"{c}_USA" for c in usa_only})

# Make sure the core common columns keep their original names
all_cols_union = sorted(list(set(cn_union.columns) | set(usa_union.columns)))
cn_union = cn_union.reindex(columns=all_cols_union)
usa_union = usa_union.reindex(columns=all_cols_union)

union_with_suffix = pd.concat([cn_union, usa_union], ignore_index=True)
print(f"Union with suffixes: {union_with_suffix.shape}")

### 3.1.4. Create Cross-Country Binary Role Variables

In [ ]:
merged["role_binary"] = np.nan  # will become 'clinician' or 'patient'

In [ ]:
# China: map therapist/client to clinician/patient
mask_cn = merged["Country"] == "China"

merged.loc[mask_cn & (merged["role_label"] == "therapist"), "role_binary"] = "clinician"
merged.loc[mask_cn & (merged["role_label"] == "client"), "role_binary"] = "patient"

In [ ]:
# USA: map 3-level role to 2-level binary
mask_usa = merged["Country"] == "USA"

merged.loc[mask_usa & (merged["role_label_usa3"] == "clinician"), "role_binary"] = "clinician"
merged.loc[mask_usa & (merged["role_label_usa3"] == "patient"), "role_binary"] = "patient"

In [ ]:
print("role_binary distribution (including NaNs):")
print(merged["role_binary"].value_counts(dropna=False))

## 3.2. Export Dataset Variants & Summary Stats

### 3.2.1. Export all dataset variants

In [ ]:
# 3.3. Export Merged Dataset Variants

merged_path = MERGED_DIR / "merged.csv"
union_path = MERGED_DIR / "union_with_suffix.csv"
intersection_path = MERGED_DIR / "intersection.csv"

merged.to_csv(merged_path, index=False)
union_with_suffix.to_csv(union_path, index=False)
intersection.to_csv(intersection_path, index=False)

print("Saved:")
print(" -", merged_path)
print(" -", intersection_path)
print(" -", union_path)

### 3.2.2 Summary stats by Country for key composites

In [ ]:
print("Summary Statistics by Country:")
for col in KEY_COMPOSITES:
    if col in merged.columns:
        print(f"\n{col}:")
        print(
            merged.groupby("Country")[col]
            .describe()[["count", "mean", "std", "min", "max"]]
        )

### 3.2.3. Total valid counts across merged dataset

In [ ]:
# Show summary of key variables
print("Key composite scores summary (merged):")
for comp in KEY_COMPOSITES:
    if comp in merged.columns:
        total_valid = merged[comp].notna().sum()
        total = len(merged)
        pct = total_valid / total * 100
        mean = merged[comp].mean()
        print(f"{comp:20} {total_valid:4}/{total} ({pct:5.1f}%) valid, M={mean:.2f}")

### 3.2.4. Dataset Visualization

In [ ]:
dataset_summary = pd.DataFrame({
    "Dataset": ["China", "USA", "Union", "Merged"],
    "Rows": [
        cn_coalesced.shape[0],
        usa_renamed.shape[0],
        union_with_suffix.shape[0],
        merged.shape[0],
    ],
    "Columns": [
        cn_coalesced.shape[1],
        usa_renamed.shape[1],
        union_with_suffix.shape[1],
        merged.shape[1],
    ],
})
print(dataset_summary)

In [ ]:
# Visualize dataset sizes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(range(len(dataset_summary)), dataset_summary["Rows"])
axes[0].set_xticks(range(len(dataset_summary)))
axes[0].set_xticklabels(dataset_summary["Dataset"], rotation=45, ha="right")
axes[0].set_ylabel("Number of Rows")
axes[0].set_title("Dataset Sizes (Participants)")
axes[0].grid(axis="y", alpha=0.3)

axes[1].bar(range(len(dataset_summary)), dataset_summary["Columns"])
axes[1].set_xticks(range(len(dataset_summary)))
axes[1].set_xticklabels(dataset_summary["Dataset"], rotation=45, ha="right")
axes[1].set_ylabel("Number of Columns")
axes[1].set_title("Dataset Widths (Variables)")
axes[1].grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("Key observations:")
print(f"Union has maximum columns ({union_with_suffix.shape[1]}) - all variables with suffixes.")
print(f"Merged has {merged.shape[1]} columns - balanced for analysis.")

# 4.0. Data Quality Summary

## 4.1. Data Quality by Country
- We print N, mean, SD for the main composites in each country.

In [ ]:
print("DATA QUALITY SUMMARY BY COUNTRY")

for country in ["China", "USA"]:
    df_country = merged[merged["Country"] == country]
    print(f"{country} (N={len(df_country)})")
    for comp in ["TENS_Life_mean", "ET_mean", "SSRPH_mean", "PHQ5_mean", "GAAIS_mean", "UTAUT_AI_mean"]:
        if comp in df_country.columns:
            n_valid = df_country[comp].notna().sum()
            mean = df_country[comp].mean()
            std = df_country[comp].std()
            print(f"{comp:20} N={n_valid:4}, M={mean:.2f}, SD={std:.2f}")
    print()

## 4.2. Reliability Analysis (Cronbach's Alpha)

Compute internal consistency reliability for all scales by country

In [ ]:
def cronbach_alpha(df, cols):

    items = df[cols].dropna()
    if len(items) == 0:
        return np.nan
    
    k = items.shape[1]
    if k < 2:
        return np.nan
    
    # Variance of each item
    item_var = items.var(axis=0, ddof=1).sum()
    
    # Variance of total score
    total_var = items.sum(axis=1).var(ddof=1)
    
    if total_var == 0:
        return np.nan
    
    alpha = (k / (k - 1)) * (1 - item_var / total_var)
    return alpha

In [ ]:
# Ensure GAAIS_neg_rev exists in both country-level dataframes
for df_ in [cn_coalesced, usa_renamed]:
    if 'GAAIS_neg' in df_.columns:
        df_['GAAIS_neg_rev'] = 8 - df_['GAAIS_neg']

# Define full scale item sets
scale_items_country = {
    "TENS_Life": ['TENS_Life_1r', 'TENS_Life_2r', 'TENS_Life_3r', 'TENS_Life_4r',
                  'TENS_Life_5r', 'TENS_Life_6r', 'TENS_Life_7', 'TENS_Life_8', 'TENS_Life_9'],
    "ET":        [f'ET_{i}' for i in range(1, 16)],
    "SSRPH":     [f'SSRPH_{i}' for i in range(1, 6)],
    "PHQ5":      [f'PHQ5_{i}' for i in range(1, 6)],
    "GAAIS_overall": ['GAAIS_pos', 'GAAIS_neg_rev'],
}

In [ ]:
print("CRONBACH'S ALPHA (FULL SCALES) BY COUNTRY")
print("=" * 60)

for country_name, df_country in [("China", cn_coalesced), ("USA", usa_renamed)]:
    print(f"{country_name}:")
    print("-" * 60)
    for scale_name, items in scale_items_country.items():
        available_items = [col for col in items if col in df_country.columns]
        if len(available_items) >= 2:
            alpha = cronbach_alpha(df_country, available_items)
            n_items = len(available_items)
            n_valid = df_country[available_items].dropna().shape[0]
            print(f"{scale_name:15} α = {alpha:.3f}  (k={n_items:2}, n={n_valid:4})")
    print()

### Important Note 
GAAIS in China is very noisy (α ≈ .06).
- Another reason to treat GAAIS clearly as a confounder, not a core moderator.
- This confirms that GAAIS (and ET) are controls, not main variables of interest.

# 5.0. Missing Data & Imputation

## 5.1. Missingness Summary for Key Variables

Missingness ranges: 4.9% (age) to ~10.7% (SSRPH_mean)

Complete cases across all h_vars: 2085 / 2342 (~89.0%)

In [ ]:
h_vars = [
    "TENS_Life_mean",    # SDT
    "UTAUT_AI_mean",     # Overall AI acceptance
    "GAAIS_mean",        # General AI attitudes
    "ET_mean",           # Epistemic trust
    "PHQ5_mean",         # Depression
    "SSRPH_mean",        # Stigma
    "age",               # Demographic covariate
    "Accept_avatar",     # Avatar / AI therapist acceptance
    "Accept_chatbot",    # Chatbot acceptance
    "Accept_tele",       # Teletherapy (human therapist) acceptance
]

print("Missing data summary before imputation:")
print("-" * 50)
for var in h_vars:
    if var in merged.columns:
        n_missing = merged[var].isna().sum()
        pct_missing = (n_missing / len(merged)) * 100
        print(f"{var:20} {n_missing:4} missing ({pct_missing:5.1f}%)")

complete_before = merged[h_vars].dropna()
print(
    f"\nComplete cases (listwise deletion across h_vars): "
    f"{len(complete_before)} / {len(merged)} ({len(complete_before)/len(merged)*100:.1f}%)"
)

## 5.2. Missing Design Matrix

- Uses gender and Country as predictors in imputation, which is reasonable and consistent with later moderator/control structure.

This is a reasonable imputation model, because it uses:
- All main continuous variables
- Key categorical predictors of missingness (gender, Country)

In [ ]:
impute_vars = h_vars + ["gender", "Country"]
impute_df = merged[impute_vars].copy()

impute_df_encoded = pd.get_dummies(impute_df, columns=["gender", "Country"], drop_first=True)

print(f"Imputation dataset shape: {impute_df_encoded.shape}")
print("Variables:", list(impute_df_encoded.columns))

## 5.3. Iterative Imputation & Storage

Make sure we have a fully processed dataset after imputation for later hypothesis testing.

### 5.3.1. Iterative Impututation (MICE-Imputation)
- This does not alter original variables; it simply creates parallel *_imputed columns for robustness checks in later hypothesis models.

In [ ]:
imputer = IterativeImputer(random_state=42, max_iter=10)
imputed_array = imputer.fit_transform(impute_df_encoded)

merged_imputed = pd.DataFrame(imputed_array, columns=impute_df_encoded.columns, index=merged.index)

for var in h_vars:
    merged[f"{var}_imputed"] = merged_imputed[var]

print("Imputation complete.")
print("Imputed variables created:")
print([f"{v}_imputed" for v in h_vars])

### 5.3.2. Compare distributions before and after imputation
- Shows that imputation does not dramatically distort distributions.
- This supports your methodological claim that imputed vs observed distributions are very similar.

In [ ]:
print("Comparison of means before/after imputation:")
print("=" * 60)
print(f"{'Variable':20} {'Original':>12} {'Imputed':>12} {'Diff':>10}")
print("-" * 60)

for var in h_vars:
    if var in merged.columns and f"{var}_imputed" in merged.columns:
        orig_mean = merged[var].mean()
        imp_mean = merged[f"{var}_imputed"].mean()
        diff = imp_mean - orig_mean
        print(f"{var:20} {orig_mean:12.3f} {imp_mean:12.3f} {diff:10.3f}")

In [ ]:
# verify no missingness remains in *_imputed variables
print("Remaining missing values in *_imputed variables:")
for var in h_vars:
    col = f"{var}_imputed"
    n_miss = merged[col].isna().sum()
    print(f"{col:20} {n_miss:4} missing")

We now have clean composites for:
- TENS_Life_mean
- UTAUT_AI_mean
- GAAIS_mean
- ET_mean
- PHQ5_mean
- SSRPH_mean
- age
- gender
- Country

Imputed versions for all continuous ones (*_imputed).
merged dataframe is already saved as CSV.

### 5.3.3. Save fully processed dataset for hypothesis testing

In [ ]:
# Create an output directory under data/
OUTPUT_DIR = DATA_DIR / "output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

processed_path = OUTPUT_DIR / "processed.csv"
merged.to_csv(processed_path, index=False)

Intervention types are now ready for separate models.
- Accept_avatar_imputed, 
- Accept_chatbot_imputed, 
- Accept_tele_imputed

Confounders first: All confounder variables and their imputed versions are present in one place in processed.csv, so we can start the Results by describing them and then build moderation models.

Moderators: Country is fully usable for moderation.

role_label is correctly defined for China; we’ll just subset Country == "China" when you run role moderation models later.